In [1]:
import numpy as np
import string
import theano
import theano.tensor as T
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
%matplotlib inline

/home/parmeet/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:280: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [2]:
def init_weight(Mi, Mo):
    return np.random.randn(Mi, Mo) / np.sqrt(Mi + Mo)

In [3]:
def remove_punctuation(s):
    return s.translate(None, string.punctuation)
def get_robert_frost():
    word2idx = {'START': 0, 'END': 1}
    current_idx = 2
    sentences = []
    for line in open('RobertFrost'):
        line = line.strip()
        if line:
            tokens = remove_punctuation(line.lower()).split()
            sentence = []
            for t in tokens:
                if t not in word2idx:
                    word2idx[t] = current_idx
                    current_idx += 1
                idx = word2idx[t]
                sentence.append(idx)
            sentences.append(sentence)
    return sentences, word2idx

In [144]:
a,b = get_robert_frost()
ar = np.array(a)

In [348]:
class rnn(object):
    def __init__(self,D,M,V):
        self.D = D
        self.M = M
        self.V = V
        print "init done"
    def generate(self,pi,word2idx):
        idx2word = {v:k for k,v in word2idx.iteritems()}
        V = len(pi)

        # generate 4 lines at a time
        n_lines = 0

        # why? because using the START symbol will always yield the same first word!
        X = [ np.random.choice(V, p=pi) ]
        print X
        print X[0]
        print idx2word[X[0]],

        while n_lines < 4:
            # print "X:", X
            P = self.predict_op(X)[-1]
            X += [P]
            if P > 1:
                # it's a real word, not start/end token
                word = idx2word[P]
                print word,
            elif P == 1:
                # end token
                n_lines += 1
                print ''
                if n_lines < 4:
                    X = [ np.random.choice(V, p=pi) ] # reset to start of line
                    print idx2word[X[0]],
    def fit(self,X,learning_rate=10e-3,mu=0.99,epochs=100,show_fig=True,f=T.nnet.sigmoid):
        ## X will be 2-D tuple
        
        
        #defining the weights
        #word embedding model
        
        V = self.V
        D = self.D
        M = self.M
        self.f = f
        We_init = init_weight(V,D)
        
        Wx_init = init_weight(D,M)
        Wh_init = init_weight(M,M)
        Wo_init = init_weight(M,V)
        bo_init = np.zeros(V)
        bh_init = np.zeros(M)
        h0_init = np.zeros(M)
        
        
        #word embedding model
        self.We = theano.shared(We_init)
        
        self.Wx = theano.shared(Wx_init)
        self.Wh = theano.shared(Wh_init)
        self.Wo = theano.shared(Wo_init)
        self.bo = theano.shared(bo_init)
        self.bh = theano.shared(bh_init)
        self.h0 = theano.shared(h0_init)
        
        self.params = [self.We,self.Wx,self.Wh,self.Wo,self.bo,self.bh,self.h0]
        
        thX = T.ivector('X')
        Ei = self.We[thX]
        #thE will be TxD matrix
        thY = T.ivector('Y')
        
        def recurrance(X,h_t):
            h_t = self.f(X.dot(self.Wx) + h_t.dot(self.Wh) + self.bh)
            y_t =  T.nnet.softmax(h_t.dot(self.Wo) + self.bo) 
            return h_t,y_t
        
        [h,y],_ = theano.scan(
                recurrance, 
                sequences=Ei, 
                outputs_info=[self.h0,None], 
                n_steps=Ei.shape[0]
        )
        
        ##py_x will be TxK matrix
        py_x = y[:,0,:]
        
        cost = -1*T.mean(T.log(py_x[T.arange(thY.shape[0]),thY]))
        grad = T.grad(cost,self.params)
        
        prediction = T.argmax(py_x,axis=1)
        
        self.dparams = [theano.shared(p.get_value()*0) for p in self.params]
        
        updates = [(p,p + mu*dp - learning_rate*g) for p,dp,g in zip(self.params,self.dparams,grad)
                  ] + [
            (dp,mu*dp - learning_rate*g) for dp,g in zip(self.dparams,grad)
        ]
        
        self.train_op = theano.function(
            inputs=[thX,thY], 
            outputs=[cost,prediction],
            updates=updates
        )
        
        self.predict_op = theano.function(
            inputs = [thX],
            outputs=prediction,
            allow_input_downcast=True,
        )
        
        costs = []
        n_total = sum(len(x) for x in X)
        for i in xrange(epochs):
            X = shuffle(X)
            cost = 0
            accuracy=0
            for j in a:
                input_sequence = [0] + j
                output_sequence = j + [1]
                c,prediction = self.train_op(input_sequence,output_sequence)
                pred = self.predict_op(input_sequence)
                #print prediction
                cost += c
                #print prediction
                #print output_sequence
                for p,out in zip(prediction,output_sequence):
                    if p==out:
                        accuracy = accuracy + 1
            print "Cost is",cost
            print "Accuracy is",(1.0*accuracy)/n_total
            costs.append(cost)
        if show_fig==True:
            plt.plot(costs)
            plt.show()
            
    

In [349]:
V = len(b.keys())

In [350]:
rn = rnn(30,30,V)

init done


In [356]:
rn.fit(a,epochs=6000,f=T.nnet.relu,learning_rate=10e-5)

Cost is 10851.8001064
Accuracy is 0.131205673759
Cost is 10021.460598
Accuracy is 0.134005225831
Cost is 9758.72912346
Accuracy is 0.132698768197
Cost is 8946.83874299
Accuracy is 0.141750653229
Cost is 8568.92703825
Accuracy is 0.137551325121
Cost is 8437.53615025
Accuracy is 0.146136618141
Cost is 8391.89870646
Accuracy is 0.152388951101
Cost is 8343.1989443
Accuracy is 0.153322135125
Cost is 8284.96541692
Accuracy is 0.149029488615
Cost is 8227.86340903
Accuracy is 0.151829040687
Cost is 8183.74405577
Accuracy is 0.151642403882
Cost is 8145.65164815
Accuracy is 0.150989175065
Cost is 8111.73256261
Accuracy is 0.149962672639
Cost is 8079.76712001
Accuracy is 0.149869354237
Cost is 8047.40434533
Accuracy is 0.149122807018
Cost is 8017.39047025
Accuracy is 0.149309443822
Cost is 7988.01116484
Accuracy is 0.149402762225
Cost is 7959.51787166
Accuracy is 0.149682717432
Cost is 7935.55587954
Accuracy is 0.149776035834
Cost is 7911.67387271
Accuracy is 0.150055991041
Cost is 7886.62192591


KeyboardInterrupt: 

In [352]:
word2idx = b

In [353]:
sentences = a
V = len(word2idx)

In [354]:
pi = np.zeros(V)
for sentence in sentences:
    pi[sentence[0]] += 1 
pi = pi/pi.sum()

In [386]:
rn.generate(pi,word2idx)

[29L]
29
the one range 
thats the voices have man 
a one descent with the step 
inscription but isnt her to be 


In [378]:
m = {value:key for key,value in word2idx.iteritems()}
m[1]

'END'

In [125]:
np.random.choice(V,pi)

ValueError: sequence too large; cannot be greater than 32

In [318]:
idx2word = {v:k for k,v in word2idx.iteritems()}

In [321]:
idx2word[[1]]

TypeError: unhashable type: 'list'